In [ ]:
from pathlib import Path
import cv2
import numpy as np
import time
import pandas as pd

OUTPUT_DIR = Path("feature_detection_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def load_or_create_image(path=None, size=(800,600)):
    """Loads image from path, or generates a synthetic textured image if not found."""
    if path:
        img = cv2.imread(path)
        if img is None:
            print(f" Could not load image at {path}. Using synthetic image instead.")
        else:
            print(f"Loaded image from {path}")
            return img

    w,h = size
    rng = np.random.RandomState(42)
    noise = (rng.rand(h,w)*255).astype(np.uint8)
    img = cv2.cvtColor(noise, cv2.COLOR_GRAY2BGR)
    img = cv2.GaussianBlur(img, (7,7), 1.5)
    cv2.putText(img, "SIFT SURF ORB", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (200,30,30), 3, cv2.LINE_AA)
    cv2.rectangle(img, (300,200), (700,450), (30,200,30), 5)
    for i in range(30):
        center = (int(rng.uniform(0,w)), int(rng.uniform(0,h)))
        radius = int(rng.uniform(5,60))
        color = tuple(int(rng.randint(0,255)) for _ in range(3))
        cv2.circle(img, center, radius, color, -1)
    print(" Generated synthetic textured image.")
    return img

def rotate_and_scale(img, angle_deg, scale):
    """Rotate and scale an image for robustness testing."""
    h,w = img.shape[:2]
    center = (w//2, h//2)
    M = cv2.getRotationMatrix2D(center, angle_deg, scale)
    return cv2.warpAffine(img, M, (w,h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

def create_sift():
    try:
        sift = cv2.SIFT_create()
        return sift, "SIFT"
    except Exception:
        try:
            sift = cv2.xfeatures2d.SIFT_create()
            return sift, "SIFT (xfeatures2d)"
        except Exception:
            print("  SIFT not available in this OpenCV build.")
            return None, None

def create_surf(hessianThreshold=400):
    try:
        surf = cv2.xfeatures2d.SURF_create(hessianThreshold)
        return surf, "SURF"
    except Exception:
        print(" SURF not available (requires opencv-contrib-python).")
        return None, None

def create_orb(nfeatures=5000):
    orb = cv2.ORB_create(nfeatures=nfeatures)
    return orb, "ORB"

def run_detector(detector, img_gray):
    if detector is None:
        return None, None, None
    t0 = time.perf_counter()
    kps, desc = detector.detectAndCompute(img_gray, None)
    t1 = time.perf_counter()
    return kps, desc, (t1 - t0)

def draw_keypoints(img, kps, outpath, max_kp_draw=5000):
    img_kp = cv2.drawKeypoints(img, kps[:max_kp_draw], None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite(str(outpath), img_kp)
    return str(outpath)

def match_descriptors(desc1, desc2, is_binary):
    if desc1 is None or desc2 is None or len(desc1)==0 or len(desc2)==0:
        return 0
    if is_binary:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    else:
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    try:
        matches = bf.knnMatch(desc1, desc2, k=2)
    except Exception:
        matches_single = bf.match(desc1, desc2)
        matches = [[m] for m in matches_single]
    good = []
    for m in matches:
        if len(m) == 2:
            if m[0].distance < 0.75 * m[1].distance:
                good.append(m[0])
        elif len(m) == 1:
            good.append(m[0])
    return len(good)

def main(image_path=None):
    img = load_or_create_image(image_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    sift, sname = create_sift()
    surf, suname = create_surf()
    orb, oname = create_orb()

    detectors = []
    if sift: detectors.append((sname, sift, False))
    if surf: detectors.append((suname, surf, False))
    if orb: detectors.append((oname, orb, True))

    results = []
    vis_paths = {}

    for name, det, is_binary in detectors:
        kps, desc, runtime = run_detector(det, img_gray)
        n_kp = 0 if kps is None else len(kps)
        desc_shape = (0,0) if desc is None else desc.shape
        desc_dtype = None if desc is None else str(desc.dtype)
        outvis = OUTPUT_DIR / f"{name.replace(' ','_')}_keypoints.png"
        vis_path = draw_keypoints(img, kps if kps else [], outvis)
        vis_paths[name] = vis_path
        results.append({
            "Method": name,
            "NumKeypoints": n_kp,
            "DescriptorShape": f"{desc_shape}",
            "DescriptorDtype": desc_dtype,
            "Runtime_s": runtime,
            "IsBinary": is_binary
        })

    transforms = [
        ("rot30_scale1.0", 30, 1.0),
        ("rot60_scale1.0", 60, 1.0),
        ("rot0_scale0.5", 0, 0.5),
        ("rot0_scale1.5", 0, 1.5),
        ("rot45_scale0.7", 45, 0.7)
    ]
    robustness_summary = []
    for name, det, is_binary in detectors:
        kps0, desc0, _ = run_detector(det, img_gray)
        good_counts = []
        for label, angle, scale in transforms:
            img_t = rotate_and_scale(img, angle, scale)
            img_t_gray = cv2.cvtColor(img_t, cv2.COLOR_BGR2GRAY)
            kps_t, desc_t, _ = run_detector(det, img_t_gray)
            matched = match_descriptors(desc0, desc_t, is_binary)
            good_counts.append(matched)
        robustness_summary.append((name, transforms, good_counts))

    df = pd.DataFrame(results).set_index("Method")
    csv_out = OUTPUT_DIR / "feature_summary.csv"
    df.to_csv(csv_out)


    report_lines = []
    report_lines.append("Short report comparing SIFT, SURF, and ORB\n")
    report_lines.append("Summary table saved to: " + str(csv_out) + "\n")
    for idx,row in df.iterrows():
        report_lines.append(f"Method: {idx}\n - Num keypoints: {row['NumKeypoints']}\n - Descriptor: {row['DescriptorShape']} dtype={row['DescriptorDtype']}\n - Runtime (s): {row['Runtime_s']:.6f}\n - Binary descriptors: {row['IsBinary']}\n")
    report_lines.append("\nRobustness (good matches counts):\n")
    for name, transforms, good_counts in robustness_summary:
        report_lines.append(f"{name}:\n")
        for (label,angle,scale), cnt in zip(transforms, good_counts):
            report_lines.append(f"  {label} (angle={angle}, scale={scale}): {cnt} good matches\n")
        report_lines.append("\n")
    report_text = "\n".join(report_lines)
    report_path = OUTPUT_DIR / "short_report.txt"
    with open(report_path, "w") as f:
        f.write(report_text)

    print("Results saved to:", OUTPUT_DIR.resolve())
    print("Summary CSV:", csv_out)
    print("Report:", report_path)
    print("\n--- Report Preview ---\n")
    print(report_text)
if __name__ == "__main__":
    imgpath = "/content/IMG.jpg"
    main(imgpath)


 Could not load image at /content/IMG.jpg. Using synthetic image instead.
 Generated synthetic textured image.
 SURF not available (requires opencv-contrib-python).
Results saved to: /content/feature_detection_outputs
Summary CSV: feature_detection_outputs/feature_summary.csv
Report: feature_detection_outputs/short_report.txt

--- Report Preview ---

Short report comparing SIFT, SURF, and ORB

Summary table saved to: feature_detection_outputs/feature_summary.csv

Method: SIFT
 - Num keypoints: 6551
 - Descriptor: (6551, 128) dtype=float32
 - Runtime (s): 0.551415
 - Binary descriptors: False

Method: ORB
 - Num keypoints: 4597
 - Descriptor: (4597, 32) dtype=uint8
 - Runtime (s): 0.044145
 - Binary descriptors: True


Robustness (good matches counts):

SIFT:

  rot30_scale1.0 (angle=30, scale=1.0): 3883 good matches

  rot60_scale1.0 (angle=60, scale=1.0): 3713 good matches

  rot0_scale0.5 (angle=0, scale=0.5): 671 good matches

  rot0_scale1.5 (angle=0, scale=1.5): 1824 good matches
